# MLSGrid MRED MLS Replication Working Notebook
## Objectives & Overview
This project will, ultimately, deploy an application to Amazon Web Services (AWS) which accomplishes the following primary objectives.
* Replicates the MRED MLS database
    1. First in an initial replication and then
    1. Maintains an updated replica of the MLS
* Provides an API to allow for data analysis using the replica

Additional discretionary objectives of this project include:
* Publishing an open source Python API wrapper for the MLSGrid API
* Publishing an open source Python package to enable the primary objectives of this project

Unfortunately, as of the time of this project, MLSGrid does not provide any Python enablement of any of this (be it API, SDK, etc...).  Hence, this project is born.

We're working through the [MLSGrid documentation](https://docs.mlsgrid.com/api-documentation/api-version-2.0) to build the Python wrapper to the MLSGrid API.  Some important notes as we start.
* MLSGrid only provide a replication API, which uses the rather stodgy and seemingly outdated ODATA standard.
* MLSGrid's support team seems thinly staffed, which has resulted in a rather "go it alone" approach to getting this done.

## Scaffolding the Project
This is going to be a bit iterative, but let's stub it out as follows:

```
mlsgrid_api
|   mred_notebook.ipynb
|   COPYING
|   README.md
|   .gitignore
|   requirements.txt
|   .env
|   mlsgrid_api.py
|___.venv (python -m venv .venv)
    |   ...venv stuff here...
|___tests
    |   test_mlsgrid_api.py
```

Let's create our project directory

    `mkdir mlsgrid_api && cd mlsgrid_api`

### #TODO
- [ ] Explore using this Jupyter notebook as the README.md for the GitHub repo
    - [ ] [Google search here](https://www.google.com/search?q=use+jupyter+notebook+as+readme.md+github&oq=use+&aqs=edge.0.69i59j69i57j0i131i433i512l2j69i60j69i65l2j69i60j69i61.1202j0j1&sourceid=chrome&ie=UTF-8)
    - [ ] [Here's an article](https://andrewpwheeler.com/2021/09/06/using-jupyter-notebooks-to-make-nice-readmes-for-github/)

We're already working in this Jupyter notebook, so make sure it's in the project directory.  In order to run any of the code in this notebook, we need to setup our Python virtual environment and install some packages.

## Python Virtual Environment
Let's create our virtual environment.  We'll do this in the shell.

    `python -m venv .venv`

Let's activate our new Python environment.

    `source /.venv/bin/activate`

Now, let's upgrade `pip` as it will complain otherwise ...

    `pip install --upgrade pip`

Let's now install `jupyterlab` to get all of the goodness.

    `pip install jupyterlab`

You can now start VSCode in the project directory.

    `code .`

If you're working in VSCode, make sure you switch the interpreter & kernel to use your newly created virtual environment in this notebook going forward.

We will publish this project publicly, opting for the **GNU General Public License v3.0** license after using [Choose A License](https://choosealicense.com/) for help.  The license is contained in the `LICENSE` file in the project directory.

In [2]:
! curl https://www.gnu.org/licenses/gpl-3.0.txt -o LICENSE

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 35149  100 35149    0     0   134k      0 --:--:-- --:--:-- --:--:--  134k


## Git Repository
Let's setup a git repo for the work we're doing so that we can manage versioning.  Before we do that, as we'll be working with sensitive API keys in our `.env` file, and we also have our virtual environment installed in the project directory (`./.venv/`), let's pull down a good Python-relevant `.gitignore` file to exclude these from the repo.  We'll use the version that [GitHub provides here](https://github.com/github/gitignore/blob/e5323759e387ba347a9d50f8b0ddd16502eb71d4/Python.gitignore).

In [8]:
! curl https://raw.githubusercontent.com/github/gitignore/main/Python.gitignore -o .gitignore

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3078  100  3078    0     0  13042      0 --:--:-- --:--:-- --:--:-- 12987


Now, we're ready to initialize our git repo.

In [9]:
! git init

Initialized empty Git repository in /home/captivus/projects/mlsgrid_api/.git/


Let's add everything we have so far to our local repo.

In [10]:
! git add .

Let's check the status of what's about to be committed to our git repo.

In [11]:
! git status

On branch master

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   .gitignore
	new file:   LICENSE
	new file:   README.md
	new file:   mred_notebook.ipynb



Now, let's commit it.

In [12]:
! git commit -m 'initial commit'

[master (root-commit) 659b78f] initial commit
 4 files changed, 992 insertions(+)
 create mode 100644 .gitignore
 create mode 100644 LICENSE
 create mode 100644 README.md
 create mode 100644 mred_notebook.ipynb


We'll move everything in the branch we're on to the `main` branch.

In [13]:
! git branch --move main

Now let's link our local repo up to our GitHub repository.  NB you'll change your repository to match the one you created at GitHub earlier.

In [14]:
! git remote add origin git@github.com:captivus/mlsgrid_api.git

With this done, let's push our local repo to the GitHub repo.

In [15]:
! git push --set-upstream origin main

Host key fingerprint is SHA256:p2QAMXNIC1TJYWeIOttrVc98/R1BUFWu3/LiyKgUfQM
+---[ECDSA 256]---+
| .o=X*+      .o.=|
|  .o=O         o |
| .  . .   E   . .|
|o     .. . .   o |
| +   . +S o.o . .|
|. . .  o++.... o.|
|   o    o.   ...+|
|  o    .   o .oo.|
| .      ... o....|
+----[SHA256]-----+
Enumerating objects: 6, done.
Counting objects: 100% (6/6), done.
Delta compression using up to 8 threads
Compressing objects: 100% (5/5), done.
Writing objects: 100% (6/6), 15.83 KiB | 3.96 MiB/s, done.
Total 6 (delta 0), reused 0 (delta 0)
To github.com:captivus/mlsgrid_api.git
 * [new branch]      main -> main
Branch 'main' set up to track remote branch 'main' from 'origin'.


Great, now we're all setup with both local and remote git repositories.

## Python Modules Needed for the Project
This will be iterative as we work through building the project, but we know we need these modules to start.
* `requests` (for interacting with the MLSGrid API)
* `python-dotenv` (to load our .env file which contains our secrets / API keys)
* `pytest` (to perform testing on the MLSGrid API wrapper we're going to build)

Let's add these to our `requirements.txt` file.  We'll use a [here document](https://wiki.bash-hackers.org/syntax/redirection#here_documents) to do this from the shell, using Jupyter's `%%bash` magic command.

In [24]:
%%bash

cat > requirements.txt << 'EOF'
requests
python-dotenv
pytest
EOF

Now, we'll install these dependencies.

In [25]:
! pip install -r requirements.txt

  Using cached python_dotenv-0.21.0-py3-none-any.whl (18 kB)
  Using cached pytest-7.1.3-py3-none-any.whl (298 kB)
  Using cached pluggy-1.0.0-py2.py3-none-any.whl (13 kB)
  Using cached py-1.11.0-py2.py3-none-any.whl (98 kB)
  Using cached iniconfig-1.1.1-py2.py3-none-any.whl (5.0 kB)
  Using cached tomli-2.0.1-py3-none-any.whl (12 kB)


Let's update our git repo and push.

In [26]:
! git commit -am 'added requirements' && git push

[main bc0c373] added requirements
 1 file changed, 299 insertions(+), 26 deletions(-)
Host key fingerprint is SHA256:p2QAMXNIC1TJYWeIOttrVc98/R1BUFWu3/LiyKgUfQM
+---[ECDSA 256]---+
| .o=X*+      .o.=|
|  .o=O         o |
| .  . .   E   . .|
|o     .. . .   o |
| +   . +S o.o . .|
|. . .  o++.... o.|
|   o    o.   ...+|
|  o    .   o .oo.|
| .      ... o....|
+----[SHA256]-----+
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 8 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 4.28 KiB | 876.00 KiB/s, done.
Total 3 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To github.com:captivus/mlsgrid_api.git
   659b78f..bc0c373  main -> main


Hrmmm ... that didn't quite do it.  Let's check git status.

In [27]:
! git status

On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	requirements.txt

nothing added to commit but untracked files present (use "git add" to track)


In [29]:
! git add --all && git commit -m 'added requirements' && git push

[main db6f5f1] added requirements
 1 file changed, 3 insertions(+)
 create mode 100644 requirements.txt
Host key fingerprint is SHA256:p2QAMXNIC1TJYWeIOttrVc98/R1BUFWu3/LiyKgUfQM
+---[ECDSA 256]---+
| .o=X*+      .o.=|
|  .o=O         o |
| .  . .   E   . .|
|o     .. . .   o |
| +   . +S o.o . .|
|. . .  o++.... o.|
|   o    o.   ...+|
|  o    .   o .oo.|
| .      ... o....|
+----[SHA256]-----+
Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 8 threads
Compressing objects: 100% (2/2), done.
Writing objects: 100% (3/3), 319 bytes | 319.00 KiB/s, done.
Total 3 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To github.com:captivus/mlsgrid_api.git
   bc0c373..db6f5f1  main -> main


There we go!

## Project Approach
We'll iterate through our primary objectives, starting first with building a Python wrapper for the MLSGrid replication API.

## Python Wrapper for the MLSGrid Replication API
We're working through the [MLSGrid API documentation here](https://docs.mlsgrid.com/api-documentation/api-version-2.0).  Let's setup the replication API.  First, we need a variable to store the API endpoint.

In [30]:
MLSGRID_API_URL = 'https://api.mlsgrid.com/v2/'

We'll store our API bearer token in our `.env` file, and export that as an environment variable using the `load_dotenv()` method of the `dotenv` module.  We'll then pull the environment variable value using `os.environ.get()`.

In [32]:
from dotenv import load_dotenv
import os

load_dotenv()
MLSGRID_API_TOKEN=os.environ.get('MLSGRID_API_TOKEN')

Now that we have our API token stored, let's setup a `requests.Session` object and parameterize the headers of it to use our bearer token in accordance with the MLSGrid documentation.

In [34]:
import requests

session = requests.Session()
session.headers.update( {'Authorization' : 'Bearer ' + MLSGRID_API_TOKEN})

We now have a requests session that we can use to query the MLSGrid API endpoint as an authorized user.  Let's test that this is working.  We'll check to see that we can access the Property entity of the API, looking only for the first result.

In [62]:
import json

response = session.get(url=MLSGRID_API_URL + 'Property?$top=1')
print(response)
print(type(response.json()))

<Response [200]>
<class 'dict'>


We see, here, that we've managed to pull a response from the MLSGrid API, with [HTTP response code 200](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/200) meaning that our request was successful!  

We also see that we can cast the response to JSON and we get a dictionary back.  We can "pretty print" that JSON dictionary as follows.

In [57]:
response.json()

{'@odata.context': 'https://api.mlsgrid.com/v2/$metadata#Property',
 'value': [{'@odata.id': "https://api.mlsgrid.com/v2/Property('MRD05253841')",
   'MRD_AAN': '847-309-1177',
   'LotSizeAcres': 0.25,
   'RoomType': ['Utility Room-1st Floor'],
   'SpecialListingConditions': ['None'],
   'InternetAddressDisplayYN': True,
   'MRD_AGE': '26-50 Years',
   'Cooling': ['Central Air'],
   'CommunityFeatures': ['Water Rights'],
   'MRD_AMT': '0',
   'MRD_AON': 'Yes',
   'Appliances': ['Range',
    'Microwave',
    'Dishwasher',
    'Refrigerator',
    'Washer',
    'Dryer'],
   'MLSAreaMajor': 'Wauconda',
   'LivingArea': 0,
   'LivingAreaSource': 'Not Reported',
   'AssociationFee': 0,
   'MRD_ATC': 'Full',
   'MRD_B78': 'Yes',
   'MRD_BAS': 'Crawl',
   'MRD_BB': 'No',
   'Basement': ['None'],
   'MRD_BOARDNUM': '2',
   'BedroomsTotal': 3,
   'BedroomsPossible': 3,
   'MRD_BRBELOW': '0',
   'MRD_CARS': '4',
   'BuyerAgencyCompensation': '2.5%-100',
   'City': 'WAUCONDA',
   'CloseDate': '200

In [65]:
parsed = json.dumps(response.json(), indent=4)
print(parsed)

{
    "@odata.context": "https://api.mlsgrid.com/v2/$metadata#Property",
    "value": [
        {
            "@odata.id": "https://api.mlsgrid.com/v2/Property('MRD05253841')",
            "MRD_AAN": "847-309-1177",
            "LotSizeAcres": 0.25,
            "RoomType": [
                "Utility Room-1st Floor"
            ],
            "SpecialListingConditions": [
                "None"
            ],
            "InternetAddressDisplayYN": true,
            "MRD_AGE": "26-50 Years",
            "Cooling": [
                "Central Air"
            ],
            "CommunityFeatures": [
                "Water Rights"
            ],
            "MRD_AMT": "0",
            "MRD_AON": "Yes",
            "Appliances": [
                "Range",
                "Microwave",
                "Dishwasher",
                "Refrigerator",
                "Washer",
                "Dryer"
            ],
            "MLSAreaMajor": "Wauconda",
            "LivingArea": 0,
            "Livi

Per the MLSGrid API documentation there are [expanded resources](https://docs.mlsgrid.com/api-documentation/api-version-2.0#resource-naming-1) that we will be querying so that they are embedded in the data returned to us.  These expanded resource names are as follows.

| Expanded Resource Name | Resources that can Expand this Resource | Description |
| ----- | ----- | -----|
| Media | Property, Member, Office | Media expandable resource. These are the media files associated with a Property, Member, or Office record. |
| Rooms | Property | Rooms expandable resource. These are the Room records associated with a Property record. |
| UnitTypes | Property | UnitTypes expandable resource. These are the UnitType records associated with a Property record. |

We'll add these expanded resources to the query we pass the API just now to make sure we're getting all of the data that we're after.

In [89]:
response = session.get(url=MLSGRID_API_URL + 'Property?$top=1&$expand=Media,Rooms,UnitTypes')
parsed = json.dumps(response.json(), indent=4)
print(parsed)
print(response.json()['value'][0]['Media'])

{
    "@odata.context": "https://api.mlsgrid.com/v2/$metadata#Property",
    "value": [
        {
            "@odata.id": "https://api.mlsgrid.com/v2/Property('MRD05253841')",
            "MRD_AAN": "847-309-1177",
            "LotSizeAcres": 0.25,
            "RoomType": [
                "Utility Room-1st Floor"
            ],
            "SpecialListingConditions": [
                "None"
            ],
            "InternetAddressDisplayYN": true,
            "MRD_AGE": "26-50 Years",
            "Cooling": [
                "Central Air"
            ],
            "CommunityFeatures": [
                "Water Rights"
            ],
            "MRD_AMT": "0",
            "MRD_AON": "Yes",
            "Appliances": [
                "Range",
                "Microwave",
                "Dishwasher",
                "Refrigerator",
                "Washer",
                "Dryer"
            ],
            "MLSAreaMajor": "Wauconda",
            "LivingArea": 0,
            "Livi

Before we commit these changes to the git repo, let's double check that our `.gitignore` file is, in fact, ignoring the secret that we've stored in our `.env` file.

In [42]:
! cat .gitignore | grep -w '^.env'

.env
venv/
venv.bak/


We're good to go, so let's commit these changes.